In [3]:
# Bibliotecas de aprendizado
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
# Bibliotecas de extracao de caracteristicas de texto
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# Bibliotecas normais
import pandas as pd, numpy

In [4]:
# carregando o dataset
dataset = pd.read_csv('dataset/good_reads_final.csv')

In [5]:
#verificando colunas
dataset.columns

Index(['author_average_rating', 'author_gender', 'author_genres', 'author_id',
       'author_name', 'author_page_url', 'author_rating_count',
       'author_review_count', 'birthplace', 'book_average_rating',
       'book_fullurl', 'book_id', 'book_title', 'genre_1', 'genre_2',
       'num_ratings', 'num_reviews', 'pages', 'publish_date', 'score'],
      dtype='object')

In [6]:
# Separacao dos conjuntos de treino e teste
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(dataset['book_title'], dataset['author_gender'], test_size = 0.2)

In [8]:
# Fazendo homem e mulher virar 0 e 1
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [11]:
# Contagem de palavras nos documentos (linhas do texto), analisa palavra por palavra, com o minimo de caracter por palavra
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
#Treina usando o texto com base nos parametros da linha anterior
count_vect.fit(dataset['book_title'])
# Transforma o conjunto de treino (titulos do livro) em bag of words
xtrain_count =  count_vect.transform(train_x)

# Transforma o conjunto de teste (titulos do livro) em bag of words
xvalid_count =  count_vect.transform(valid_x)

In [13]:
# contagem de palavras com uma analise de importancia do termo em vista do titulo 
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
#Treina usando o texto com base nos parametros da linha anterior
tfidf_vect.fit(dataset['book_title'])

# Transforma o conjunto de treino (titulos do livro) em TFIDF
xtrain_tfidf =  tfidf_vect.transform(train_x)
# Transforma o conjunto de teste (titulos do livro) em TFIDF
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [15]:
# Classifier -> Classificador que você passa, 
# Feature_vector_train -> titulos dos livros da base de treino ja modificados para numero (bag of words ou tfidf)
# label -> Rotulo do titulo do livro (genero do autor, se eh homem ou mulher)
# feature_vector_valid -> conjunto de teste (titulos dos livros ja modificados tambem)
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # Treina usando o modelo passado (conjunto de treino, rotulo do treino)
    classifier.fit(feature_vector_train, label)
    
    # Prediz os rotulos do conjunto de teste
    predictions = classifier.predict(feature_vector_valid)

    # retorna a acuracia
    return metrics.accuracy_score(predictions, valid_y)

In [16]:
#Usando Naive Bayes como algoritmo de Aprendizado
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print(accuracy)

0.6634636383489845


In [17]:
#Usando Regressao logistica como algoritmo de Aprendizado

accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

LR, WordLevel TF-IDF:  0.6689233457086701


/opt/anaconda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
# Usando Floresta Randomica como algoritmo de Aprendizado
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

/opt/anaconda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, WordLevel TF-IDF:  0.7248307490718497
